In [1]:
import torch
import torch.nn.functional as F
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

def ssim(img1, img2):
    C1 = (0.01 * 255) ** 2
    C2 = (0.03 * 255) ** 2

    mu1 = img1.mean(dim=(1, 2), keepdim=True)
    mu2 = img2.mean(dim=(1, 2), keepdim=True)
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = img1.var(dim=(1, 2), unbiased=False, keepdim=True)
    sigma2_sq = img2.var(dim=(1, 2), unbiased=False, keepdim=True)
    sigma12 = torch.mean(img1 * img2, dim=(1, 2), keepdim=True) - mu1_mu2

    ssim_numerator = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
    ssim_denominator = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)

    ssim_score = ssim_numerator / ssim_denominator
    return ssim_score

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
    psnr = 10 * torch.log10(1 / mse)
    return psnr

batch_size = gt.size(0)

# 将gt和decoded转换为浮点数类型
# gt = gt.float()
# decoded = decoded.float()

# 计算SSIM
ssim_score = ssim(decoded, gt)
# 计算平均SSIM
mean_ssim = ssim_score.mean()

# 计算PSNR
psnr_score = psnr(decoded, gt)
# 计算平均PSNR
mean_psnr = psnr_score.mean()

print("Mean SSIM:", mean_ssim.item())
print("Mean PSNR:", mean_psnr.item())

Mean SSIM: 0.9745087623596191
Mean PSNR: 5.617281913757324


In [7]:
import torch
import torch.nn.functional as F
decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
    gtmax = torch.max(gt)
    demax = torch.max(decoded)
    maxx = torch.max(gtmax,demax)
    psnr = 10 * torch.log10(maxx*maxx / mse)
    return psnr

print(f'size:{gt.shape}')
batch_size = gt.size(0)

# 计算PSNR
psnr_score = psnr(decoded, gt)
# 计算平均PSNR
mean_psnr = psnr_score.mean()

# print("Mean SSIM:", mean_ssim.item())
print(f"PSNR:{psnr_score:.4f} , Mean PSNR:{mean_psnr.item():.4f}" )

size:torch.Size([2, 361, 720])
PSNR:27.4316 , Mean PSNR:27.4316


In [14]:
import torch
import torch.nn.functional as F

decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2, reduction='mean')
    gtmax = torch.max(gt)
    demax = torch.max(decoded)
    maxx = torch.max(gtmax, demax)
    psnr = 10 * torch.log10(maxx * maxx / mse)
    return psnr
    

print(f'size:{gt.shape}')
batch_size = gt.size(0)
print(f'batchsize:{batch_size}')

# 计算每张图像的PSNR
psnr_scores = []
for i in range(batch_size):
    psnr_score = psnr(decoded[i], gt[i])
    psnr_scores.append(psnr_score.item())

# 计算平均PSNR
mean_psnr = sum(psnr_scores) / batch_size

# 输出每张图像的PSNR和平均PSNR
for i in range(batch_size):
    print(f"PSNR of Image {i+1}: {psnr_scores[i]:.4f}")
print(f"Mean PSNR: {mean_psnr:.4f}")

size:torch.Size([2, 361, 720])
batchsize:2
PSNR of Image 1: 26.3136
PSNR of Image 2: 28.9411
Mean PSNR: 27.6273


In [17]:
import torch
import torch.nn.functional as F

decoded = torch.load('decoded2.pt').cpu()
gt = torch.load('gt2.pt').cpu()

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2, reduction='mean')
    gtmax = torch.max(gt)
    demax = torch.max(decoded)
    maxx = torch.max(gtmax, demax)
    psnr = 10 * torch.log10(maxx * maxx / mse)
    return psnr
    
def batch_psnr(img1, img2):
    batch_size = img1.size(0)
    psnrlist=[]
    for i in range(batch_size):
        psnrr=psnr(img1[i],img2[i])
        psnrlist.append(psnrr)
    return torch.tensor(psnrlist)

print(f'size:{gt.shape}')
# 计算每张图像的PSNR
psnr_score = batch_psnr(decoded, gt)
print(psnr_score.shape)
# 计算平均PSNR
mean_psnr = sum(psnr_score) / batch_size

# 输出每张图像的PSNR和平均PSNR

print(f"PSNR: {psnr_score}")
print(f"Mean PSNR: {mean_psnr:.4f}")

size:torch.Size([2, 361, 720])
torch.Size([2])
PSNR: tensor([26.3136, 28.9411])
Mean PSNR: 27.6273


In [35]:
import torch
import torch.nn.functional as F

decoded = torch.load('decoded2.pt').cpu() #decoded.shape=torch.size([2,361,720])
# decoded=decoded[0].unsqueeze(0)
gt = torch.load('gt2.pt').cpu() #gt.shape=torch.size([2,361,720])
# gt = gt[0].unsqueeze(0)

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2, reduction='mean') #0.3549
    gtmax = torch.max(img1)
    demax = torch.max(img2) #1.7787
    maxx = torch.max(gtmax, demax)
    psnr = 10 * torch.log10(maxx * maxx / mse)
    return psnr
    
def batch_psnr(img1, img2):
    batch_size = img1.size(0)
    psnrlist=[]
    for i in range(batch_size):
        psnrr=psnr(img1[i],img2[i])
        psnrlist.append(psnrr)
    return torch.tensor(psnrlist)

print(f'gt.shape:{gt.shape}')
psnr_score = batch_psnr(decoded, gt)# 计算每张图像的PSNR
print(f'pscr.shape:{psnr_score.shape}')
mean_psnr = sum(psnr_score) / psnr_score.__len__()# 计算平均PSNR

print(f"PSNR: {psnr_score}")
print(f"Mean PSNR: {mean_psnr:.4f}")

size:torch.Size([2, 361, 720])
torch.Size([2])
PSNR: tensor([ 9.5015, 28.9411])
Mean PSNR: 19.2213


In [40]:
import torch
import torch.nn.functional as F

decoded = torch.load('decoded2.pt').cpu() # decoded.shape=torch.Size([2, 361, 720])
gt = torch.load('gt2.pt').cpu() # gt.shape=torch.Size([2, 361, 720])

def ssim(img1, img2, window_size=11, size_average=True):
    gtmax = torch.max(img1)
    demax = torch.max(img2) #1.7787
    maxx = torch.max(gtmax, demax)
    C1 = (0.01 * maxx) ** 2
    C2 = (0.03 * maxx) ** 2

    def gaussian(window_size, sigma):
        gauss = torch.exp(-(torch.arange(window_size) - window_size // 2) ** 2 / (2 * sigma ** 2))
        return gauss / gauss.sum()

    def create_window(window_size, channel):
        _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
        _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
        window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
        return window

    channel = img1.size(0)
    window = create_window(window_size, channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size//2, groups=channel) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

def batch_ssim(img1, img2):
    batch_size = img1.size(0)
    ssim_list = []
    for i in range(batch_size):
        ssim_val = ssim(img1[i].unsqueeze(0), img2[i].unsqueeze(0))
        ssim_list.append(ssim_val)
    return torch.tensor(ssim_list)

decoded = decoded.unsqueeze(1)  # [batch_size, 1, height, width]
gt = gt.unsqueeze(1)  # [batch_size, 1, height, width]

print(f'gt.shape: {gt.shape}')
ssim_score = batch_ssim(decoded, gt) # 计算每张图像的SSIM
print(f'ssim_score.shape: {ssim_score.shape}')
mean_ssim = ssim_score.mean() # 计算平均SSIM

print(f"SSIM: {ssim_score}")
print(f"Mean SSIM: {mean_ssim:.4f}")

gt.shape: torch.Size([2, 1, 361, 720])
ssim_score.shape: torch.Size([2])
SSIM: tensor([0.4359, 0.7177])
Mean SSIM: 0.5768


In [42]:
import torch
import torch.nn.functional as F

decoded = torch.load('decoded2.pt').cpu() # decoded.shape=torch.Size([2, 361, 720])
gt = torch.load('gt2.pt').cpu() # gt.shape=torch.Size([2, 361, 720])

def ssim(img1, img2, window_size=11, size_average=False):
    img1 = img1.unsqueeze(1) # [batch_size, 1, height, width]
    img2 = img2.unsqueeze(1) # [batch_size, 1, height, width]
    gtmax = torch.max(img1)
    demax = torch.max(img2) #1.7787
    maxx = torch.max(gtmax, demax)
    C1 = (0.01 * maxx) ** 2
    C2 = (0.03 * maxx) ** 2

    def gaussian(window_size, sigma):
        gauss = torch.exp(-(torch.arange(window_size) - window_size // 2) ** 2 / (2 * sigma ** 2))
        return gauss / gauss.sum()

    def create_window(window_size, channel):
        _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
        _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
        window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
        return window

    channel = img1.size(0)
    window = create_window(window_size, channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size//2, groups=channel) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

def batch_ssim(img1, img2):
    batch_size = img1.size(0)
    ssim_list = []
    for i in range(batch_size):
        ssim_val = ssim(img1[i].unsqueeze(0), img2[i].unsqueeze(0))
        ssim_list.append(ssim_val)
    return torch.tensor(ssim_list)

# decoded = decoded.unsqueeze(1)  # [batch_size, 1, height, width]
# gt = gt.unsqueeze(1)  # [batch_size, 1, height, width]

print(f'gt.shape: {gt.shape}')
ssim_score = batch_ssim(decoded, gt) # 计算每张图像的SSIM
print(f'ssim_score.shape: {ssim_score.shape}')
mean_ssim = ssim_score.mean() # 计算平均SSIM

print(f"SSIM: {ssim_score}")
print(f"Mean SSIM: {mean_ssim:.4f}")

gt.shape: torch.Size([2, 361, 720])
ssim_score.shape: torch.Size([2])
SSIM: tensor([0.4359, 0.7177])
Mean SSIM: 0.5768


In [1]:
a=[0.1,1]
sum(a)/len(a)

0.55